# Neural Networks CW1

## Approach
This courswork follows the 'universal approach of machine learning' by Cholet. The approach can be assembled by the following steps:
1. Defining the problem and assembling the data
2. Choosing a measure of success
3. Defining an evaluation protocol
4. Preparing the data
5. Developing a model
6. Scaling - Ovefit the Model
7. Regularize model and hyperparamter tuning

## Introduction

Based on my personal interest in digital health, the question, how to leverage medcinine often comes up. At the times of ChatGPT the term AI is present everywhere. Especially in the field of modern medicin, physicians and scientis search approaches to upscale medicine and make it more efficient. The key problem of scaling up is that one physician can take care of one patient at the time. But what would be if multiple patients could be helped at the same time. That is the point where AI comes along as all healing solution. Following this path forcibly it has to proven how trustworthy these kind of solutions are. Personally I can image that the fields of A.I. and machine learning are going to enter medicine step by step in minor fields to achieve a higher grade of automation e.g. evaluating X-ray images.


## 1. Definition of the Problem

In this coursework I would like to build a neural network to predict the presents of a heart disease. As input data serves a dataset downloaded from Kaggle. The dataset is called 'Heart Failure Prediction Dataset'. The aim is to predict, based on the given data, whether a heart disease is present. This results in the problem of binary classification. In this context, the hypothesis is: Based on the given medical observations like 'resting blood preasure' and 'chest pain type' it can be predicted if a heart disease is present.



## 2. Defining a Measure of Success

In order to evaluate the results of the model built in this notebook a measure of success needs to be defined. As mentioned in the previous section the aim is to solve a binary classification model. In dependence of the data distribution accuracy and area under the receiver operating characteristics curve (ROC, AUC) are suitable metrics.

Particularly the ROC, AUC is a good measure of success. The ROC (receiver operating characteristics curve) is a graphical representation to evaluate the binary classifier. To create the ROC the true positive rate (TPR) and false positive rate (FPR) is needed. The curve for the ROC is created by plotting TPR, FPR against each other.

An additional measure for ROC is the 'Area Under the Curve' (AUC). It measures the entire two-dimensional area under the ROC. It is measure of all possible classification tresh holds. An AUC of 1 would represent a perfect classifier while an AUC of 0.5 would represent a useless classifier.

The higher the ROC to th upper left corner, the higher is the overall accuracy. A line with a roughly 45 degree diagonal would represent a classifier with very limited discrimination ability.


## 3. Defining an evaluation protocol

## Loading the Dataset

In [32]:
# importing necessary libraries
import pandas as pd

In [33]:
# defining relative path to file
file= './data/heart.csv'

# read file as pandas dataframe
df= pd.read_csv(file, sep=',', header= 0) # define seperator as ','

In [34]:
# validation that data was load correctly
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [35]:
# check the columns of the dataframe
columns= df.columns
print(columns)
print(f'Number of columns: {len(columns)}')

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')
Number of columns: 12


The column above shows the columns of the data frame. In total the dataframe has 12 columns. The column 'HeartDisease' is the target column that is supposed.
In the following step the divided in feature columns and the target column.

In [36]:
# divide dataframe into featues and labels
df_train = df.iloc[:, :len(df.columns)-1]
df_label= df.iloc[:,-1]

In [37]:
df_train.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up


In [38]:
df_label.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [39]:
# Check the quality of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


The datasets consists of 918 observations. On first sight it seems to be that there a no missing values.

In [40]:
# Show the class distribution of the target column
df.HeartDisease.value_counts(normalize= True)

HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64

The target class is distrbuted in 55% observations that related ti a heart disease and 44% without heart disease. This gets quite close to an almost equal distribution. So the target class is well balanced.

In [54]:
# shows descriptive statistics for numeric columns
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


The describe function of the dataframe gives a statical overview of the numeric columns. Firstly there are no missing values. Secondly it can be noted that the scale of the data has big differences. This becomes clear comparing values like 'Age' and 'Cholesterol'. This shows scaling gets necessary.

In [10]:
# Checking for null values
df_train.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
dtype: int64

In [11]:
# Scaling numeric values
num_cols= ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

In [12]:
# Following the numeric coloumns of the datafram will be scaled
for col in num_cols:
    df_train[col] = (df_train[col] - df_train[col].mean()) / df_train[col].std()


In [31]:
# Transform categorical variable into dummy variables; source: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
cat_cols= ['Sex', 'ChestPainType','RestingECG','ExerciseAngina', 'ST_Slope']
df_train= pd.get_dummies(df_train, columns=cat_cols)

KeyError: "None of [Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')] are in the [columns]"

Hence applied the 'get_dummies' method from pandas the categorical data is is split in n boolean columns representing the number of unique values. Corresponding to the distritbution of the the orginial values. A appearance of the categorical value is stored as boolean value. The aim ist to change the boolean values into ones and zeros representing the boolean value.

In [22]:
cat_cols= ['Sex_F', 'Sex_M', 'ChestPainType_ASY', 'ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA', 'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST', 'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Down', 'ST_Slope_Flat', 'ST_Slope_Up']


In [27]:
df_train[cat_cols]= df_train[cat_cols].astype(str)

In [24]:
arr_train= df_train.to_numpy()
arr_train[:10]

array([[-1.4323590105189488, 0.41068502452424355, 0.824620753758324,
        -0.5510409629061659, 1.3821747828285678, -0.8319788750413097,
        'False', 'True', 'False', 'True', 'False', 'False', 'False',
        'True', 'False', 'True', 'False', 'False', 'False', 'True'],
       [-0.4782229027299014, 1.490939616252769, -0.1718673610980285,
        -0.5510409629061659, 0.7537462668036182, 0.10560596081422968,
        'True', 'False', 'False', 'False', 'True', 'False', 'False',
        'True', 'False', 'True', 'False', 'False', 'True', 'False'],
       [-1.7504043797819644, -0.1294422713400192, 0.7697681969772403,
        -0.5510409629061659, -1.524307103786824, -0.8319788750413097,
        'False', 'True', 'False', 'True', 'False', 'False', 'False',
        'False', 'True', 'True', 'False', 'False', 'False', 'True'],
       [-0.5842380258175733, 0.302659565351391, 0.1389637939947787,
        -0.5510409629061659, -1.1315392812712306, 0.5743983787419994,
        'True', 'False', 'True

In [14]:
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np

2023-12-05 19:45:12.439280: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def build_model(learning_rate= .1, loss= 'binary_crossentropy', activation='relu'):
    model= models.Sequential()
    model.add(layers.Dense())

# References

- Dataset: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction